In [ ]:
from dataclasses import dataclass, replace
import pickle
from IPython.display import display
import ipywidgets as widgets
import cvxopt
import numpy as np
from matplotlib import pyplot
from donotation import do
import statemonad
import polymat, polymat.typing
import sosopt, sosopt.typing

In [ ]:
# Initialize state object
state = polymat.init_state()

# 2. Power Converter Case Study

In this case study, the input constraints of the power converter model are disregarded.

## Introduction

In this notebook, we compute a Control Lyapunov Function (CLF) and two Control Barrier Functions (CBFs) for a safety filter applied to a power converter model without input constraints using an alternating algorithm.

1. The notebook begins by defining a dynamical system model using polynomial matrices and specifying the requirements for the CLF and CBF.
2. Next, it generates an initial set of values to initialize the alternating algorithm, and proceeds with the implementation of the algorithm. 
3. Finally, the results are visualized through plots and saved in a pickle fil, enabling further simulation and analysis.

<!-- ## State

We begin by initializing a state object, which stores itermediate computational results to ensure repoducibility.
This state object is passed through all operations related to solving the SOS problem.

An architecture utilizing a state object offers flexibility for converting the data between different representations and efficiently serializing computational results. -->

## Dynamical Model

The following subsection defines the dynamical system model of the power converter case study using polynomial matrices.

In [ ]:
@dataclass
class ModelParam:
    s_n: float
    w_n: float

    v_grid_phph: float
    r_grid_si: float
    l_grid_si: float

    r_tr: float
    x_tr: float

    c_dc_si: float
    r_dc_si: float
    v_dc_n: float

    def __post_init__(self):
        self.v_n = self.v_grid_phph / np.sqrt(3)
        self.i_n = self.s_n / self.v_n
        self.z_n = self.v_n**2 / self.s_n
        self.l_n = self.z_n / self.w_n
        self.z_dc_n = self.v_dc_n**2 / self.s_n
        self.c_dc_n = 1 / (self.z_dc_n * self.w_n)

        self.r_grid = self.r_grid_si / self.z_n
        self.x_grid = self.l_grid_si / self.l_n
        self.c_dc = self.c_dc_si / self.c_dc_n
        self.r_dc = self.r_dc_si / self.z_dc_n

        self.g_dc = 1 / self.r_dc

        self.l = self.x_tr + self.x_grid


model = ModelParam(
    s_n=20e6 / 3,
    w_n=2 * np.pi * 50,
    v_grid_phph=130e3,
    r_grid_si=1e-3,
    l_grid_si=0.01,
    r_tr=2 * 3e-3,
    x_tr=2 * 9e-2,
    c_dc_si=2 * 10e-3,
    r_dc_si=1000,
    v_dc_n=2400,
)

# steady state input offset
u0 = np.array(((1, -model.l*model.g_dc),)).T

In [ ]:
# polynomial degrees
####################

u_degrees = (0, 1, 2, 3)
V_degrees = (0, 1, 2, 3, 4)
B_degrees = (1, 2, 3, 4)


# variables
###########

# define state variables
variable_names = ("v_dc", "i_d", "i_q")
state_variables = tuple(polymat.define_variable(name) for name in variable_names)
v_dc, i_d, i_q = state_variables
x = polymat.v_stack(state_variables)

n_states = len(state_variables)
n_inputs = 2


# system model
##############

# The dynamical system model is given by x_dot := G @ x + f @ u

scale = polymat.from_((
    (1 / model.c_dc, 1 / model.l, 1 / model.l),
)).diag() * model.w_n

f = scale @ polymat.from_((
    (-model.g_dc*v_dc - i_d + model.g_dc*model.l*i_q,),
    (v_dc + model.l*i_q,),
    (-model.g_dc*model.l*v_dc - model.l*i_d,),
))

G = scale @ polymat.from_((
    (model.g_dc-i_d, -i_q), 
    (1+v_dc, 0), 
    (0, 1+v_dc),
))


# state feedback controller
###########################

# The state feedback controller is defined as the reational function u(x) := p(x) / s(x)

p_monom = x.combinations(degrees=u_degrees)
p = sosopt.define_polynomial(
    name="p", 
    monomials=p_monom, 
    polynomial_variables=x,
    n_row=n_inputs,
)

G_at_p = (G @ p).cache()

# compute the degree of polynomial s
state, max_degrees = polymat.to_degree(G_at_p, variables=x).apply(state)
max_degree: int = int(np.max(max_degrees))

state, s = sosopt.define_multiplier(
    name="s",
    degree=max_degree,
    multiplicand=f,
    variables=x,
).apply(state)

x_dot = (s * f + G_at_p).cache()


# nominal controller
####################

u_n = polymat.from_(np.array((
    ((0.1*v_dc - i_d),),
    ((0 - i_q),),
)))

x_n_dot = f + G @ u_n


# state constraints
###################

# Define the state constraints as the intersection of zero-sublevel sets of two polynomials, w1 and w2
w1 = ((v_dc + 0.3) / 0.5) ** 2 + (i_d / 20) ** 2 + (i_q / 20) ** 2 - 1
w2 = ((v_dc + 0.3) / 20) ** 2 + (i_d / 1.3) ** 2 + (i_q / 1.3) ** 2 - 1

## Specifications

This subsection specifies the conditions required for both the CLF and CBF.

In [ ]:
# Control Lyapunov and Barrier Functions
########################################

V_monom = x.combinations(degrees=V_degrees)
V = sosopt.define_polynomial(
    name="V", 
    monomials=V_monom, 
    polynomial_variables=x,
)
dV = V.diff(x).T.cache()

B1_monom = x.combinations(degrees=B_degrees)
B1_var = sosopt.define_polynomial(
    name="B1", 
    monomials=B1_monom, 
    polynomial_variables=x,
)
B1 = B1_var - 1
dB1 = B1.diff(x).T.cache()

B2_monom = x.combinations(degrees=B_degrees)
B2_var = sosopt.define_polynomial(
    name="B2", 
    monomials=B2_monom, 
    polynomial_variables=x,
)
B2 = B2_var - 1
dB2 = B2.diff(x).T.cache()

# margins used for solving the bilinear problem using an alternating algorithm
clf_epsilon = sosopt.define_variable(name="clf_epsilon")
cbf1_epsilon = sosopt.define_variable(name="cbf1_epsilon")
cbf2_epsilon = sosopt.define_variable(name="cbf2_epsilon")


# Region Of Interest (ROI)
##########################

# The region of interest is defined by the zero sublevel set of the following polynomials
# It is selected to contain the allowable set X_a.
roi = (
    1.0 - (0.259537750205392 * i_d**2 + 0.259537750205392 * i_q**2 + 3.50960314067544 * v_dc**2 + 2.10573558538538 * v_dc)
).cache()

# Increase the zero sublevel set of the region of interest by decreasing epsilon from 1 to 0
epsilon_roi_V = sosopt.define_variable(name="epsilon_roi_V")
epsilon_roi_B = sosopt.define_variable(name="epsilon_roi_B")

roi_V = roi - epsilon_roi_V
roi_B = roi - epsilon_roi_B


# Dissipation rate
##################

dissipation_rate = 0.01 * s * roi * (V + 1)

In [ ]:
@do()
def define_constraints():

    # Control Lyapunov Function (CLF) condition
    clf_condition = yield from sosopt.sos_constraint_putinar(
        name="clf",
        greater_than_zero=-(dV.T @ x_dot) - dissipation_rate + clf_epsilon,
        domain=sosopt.set_(
            greater_than_zero={"V": V, "roi": roi_V},
        ),
    )

    # Forward invariance condition of the nominal region w.r.t. the nominal controller.
    clf_unom_condition = yield from sosopt.sos_constraint_putinar(
        name="unom",
        greater_than_zero=-(dV.T @ x_n_dot) - dissipation_rate,
        domain=sosopt.set_(
            greater_than_zero={"roi": roi_V},
            equal_zero={"V": V},
        ),
    )

    # Control Lyapunov Function (CBF) condition
    cbf1_condition = yield from sosopt.sos_constraint_putinar(
        name="cbf1",
        greater_than_zero=-(dB1.T @ x_dot) + cbf1_epsilon,
        domain=sosopt.set_(
            greater_than_zero={"roi": roi_B},
            equal_zero={"B": B1},
        ),
    )

    cbf2_condition = yield from sosopt.sos_constraint_putinar(
        name="cbf2",
        greater_than_zero=-(dB2.T @ x_dot) + cbf2_epsilon,
        domain=sosopt.set_(
            greater_than_zero={"roi": roi_B},
            equal_zero={"B": B2},
        ),
    )

    # These conditions ensure that the safety set contains nominal region
    b1_contains_v = yield from sosopt.sos_constraint(
        name="b1v",
        greater_than_zero=V - B1,
    )

    b2_contains_v = yield from sosopt.sos_constraint(
        name="b2v",
        greater_than_zero=V - B2,
    )

    # These conditions ensure that the safety set is contained in allowable region
    w1_contains_b1 = yield from sosopt.sos_constraint_putinar(
        name="b1w1",
        greater_than_zero=B1,
        domain=sosopt.set_(
            greater_than_zero={"w": w1},
        ),
    )

    w2_contains_b2 = yield from sosopt.sos_constraint_putinar(
        name="b2w2",
        greater_than_zero=B2,
        domain=sosopt.set_(
            greater_than_zero={"w": w2},
        ),
    )

    # positivity condition of the denominator
    s_positive = yield from sosopt.sos_constraint(
        name="spos",
        greater_than_zero=s - 0.001,
    )

    # This positivity condition ensures that the region of interest grows only as much as necessary
    epsilon_roiV = yield from sosopt.sos_constraint(
        name="epsilon_roiV",
        greater_than_zero=epsilon_roi_V,
    )

    epsilon_roiB = yield from sosopt.sos_constraint(
        name="epsilon_roiB",
        greater_than_zero=epsilon_roi_B,
    )

    # Limit CLF/CBF condition margins for numerical stability
    min_margin = -0.01
    epsilon_clf = yield from sosopt.sos_constraint(
        name="epsilon_clf",
        greater_than_zero=clf_epsilon - min_margin,
    )
    
    epsilon_cbf1 = yield from sosopt.sos_constraint(
        name="epsilon_cbf1",
        greater_than_zero=cbf1_epsilon - min_margin,
    )
    
    epsilon_cbf2 = yield from sosopt.sos_constraint(
        name="epsilon_cbf2",
        greater_than_zero=cbf2_epsilon - min_margin,
    )

    constraints = (
        clf_condition,
        clf_unom_condition,
        cbf1_condition,
        cbf2_condition,

        w1_contains_b1,
        w2_contains_b2,
        b1_contains_v,
        b2_contains_v,
        
        s_positive,
        
        epsilon_roiV,
        epsilon_roiB,
        
        epsilon_clf,
        epsilon_cbf1,
        epsilon_cbf2,
    )
    return statemonad.from_({c.name: c for c in constraints})

state, constraints = define_constraints().apply(state)

## Initialization

This subsection generates an initial set of values to initialize the alternating algorithm.

In [ ]:
@do()
def get_initial_symbol_values():
    # initial nominator polynomial of the state feedback controller
    p0 = (polymat.from_((
        (-1, -1, -1),
        (-1, -1, -1),
    )) @ x)

    # initialize the algorithm with an initial guess
    init_values = (
        (p, p0),
        (s, 100),
        (constraints['clf'].multipliers["V"], 100),
        (constraints['unom'].multipliers["V"], 100),
        (constraints['cbf1'].multipliers["B"], 100),
        (constraints['cbf2'].multipliers["B"], 100),
        (epsilon_roi_V, 1),
        (epsilon_roi_B, 1),
    )

    def to_tuple():
        for expr, value_expr in init_values:
            if isinstance(value_expr, (float, int)):
                value_expr = polymat.from_vector(value_expr)

            @do()
            def to_symbol_data_tuple(expr, symbol, monomials=None):
                data = yield from polymat.to_tuple(
                    expr.linear_in(variables=x, monomials=monomials)
                )
                return statemonad.from_((symbol, data[0]))                
            
            match expr:
                case sosopt.typing.PolynomialVariable(monomials=monomials):
                    for (row, col), param in expr.iterate_coefficients():
                        yield to_symbol_data_tuple(
                            expr=value_expr[row, col],
                            symbol=param.symbol,
                            monomials=monomials,
                        )

                case polymat.typing.VariableExpression() as var_expr:
                    yield to_symbol_data_tuple(
                        expr=value_expr,
                        symbol=var_expr.symbol,
                    )

    values_tuple = yield from statemonad.zip(to_tuple())
    return statemonad.from_(dict(values_tuple))

# If True, load the initialization from a pickle file
load_from_file = False

if not load_from_file:
    state, inital_symbol_values = get_initial_symbol_values().apply(state)

else:
    file_name = '2_symbol_values.p'
    with open(file_name, 'rb') as file:   
        inital_symbol_values = pickle.load(file)

## Alternating Algorithm

This subsection defines the alternating algorithm.

In [ ]:
# data collected at each iteration of the algorithm
###################################################

@dataclass
class IterationData:
    state: polymat.typing.State
    symbol_values: dict
    solver_data: sosopt.typing.SolverData | None

iter_data = IterationData(state=state, symbol_values=inital_symbol_values, solver_data=None)

In [ ]:
# define steps/alternations of the algorithm
############################################

@dataclass
class Step:
    lin_cost: polymat.typing.MatrixExpression
    quad_cost: polymat.typing.MatrixExpression | None
    substitutions: tuple
    override_symbol_values: dict

def init_step(lin_cost, substitutions, quad_cost=None, override_symbol_values={}):
    return Step(
        lin_cost=lin_cost, quad_cost=quad_cost, substitutions=substitutions, 
        override_symbol_values=override_symbol_values,
    )

# Set CLF/CBF margins to zero when they are not being optimizied
set_epsilon_to_zero = {
    clf_epsilon.symbol: (0,),
    cbf1_epsilon.symbol: (0,),
    cbf2_epsilon.symbol: (0,),
}

step_1_margin = init_step(
    lin_cost=clf_epsilon + cbf1_epsilon + cbf2_epsilon,
    substitutions=(
        epsilon_roi_V, epsilon_roi_B, 
        p, s,
        constraints['clf'].multipliers["V"],
        constraints['unom'].multipliers["V"],
        constraints['cbf1'].multipliers["B"],
        constraints['cbf2'].multipliers["B"],
    )
)

step_1_roiV = init_step(
    lin_cost=epsilon_roi_V,
    substitutions=(
        epsilon_roi_B, 
        p, s,
        constraints['clf'].multipliers["V"], constraints['clf'].multipliers["roi"],
        constraints['unom'].multipliers["V"], constraints['unom'].multipliers["roi"],
        constraints['cbf1'].multipliers["B"],
        constraints['cbf2'].multipliers["B"],
        clf_epsilon, cbf1_epsilon, cbf2_epsilon,
    ),
    override_symbol_values=set_epsilon_to_zero,
)

step_1_roiB = init_step(
    lin_cost=epsilon_roi_B,
    substitutions=(
        epsilon_roi_V,
        p, s,
        constraints['clf'].multipliers["V"],
        constraints['unom'].multipliers["V"],
        constraints['cbf1'].multipliers["B"], constraints['cbf1'].multipliers["roi"],
        constraints['cbf2'].multipliers["B"], constraints['cbf2'].multipliers["roi"],
        clf_epsilon, cbf1_epsilon, cbf2_epsilon,
    ),
    override_symbol_values=set_epsilon_to_zero,
)

step_1_vol = init_step(
    lin_cost=sosopt.to_gram_matrix(B1, x).trace() + sosopt.to_gram_matrix(B2, x).trace(),
    substitutions=(
        epsilon_roi_V, epsilon_roi_B,
        p, s,
        constraints['clf'].multipliers["V"],
        constraints['unom'].multipliers["V"],
        constraints['cbf1'].multipliers["B"],
        constraints['cbf2'].multipliers["B"],
        clf_epsilon, cbf1_epsilon, cbf2_epsilon,
    ),
    override_symbol_values=set_epsilon_to_zero,
)

step_2_margin = init_step(
    lin_cost=clf_epsilon + cbf1_epsilon + cbf2_epsilon,
    substitutions=(
        epsilon_roi_V, epsilon_roi_B, 
        V, B1_var, B2_var,
    )
)

step_2_roiV = init_step(
    lin_cost=epsilon_roi_V,
    substitutions=(
        epsilon_roi_B, 
        V, B1_var, B2_var,
        constraints['clf'].multipliers["roi"],
        constraints['unom'].multipliers["roi"],
        clf_epsilon, cbf1_epsilon, cbf2_epsilon,
    ),
    override_symbol_values=set_epsilon_to_zero,
)

step_2_roiB = init_step(
    lin_cost=epsilon_roi_B,
    substitutions=(
        epsilon_roi_V, 
        V, B1_var, B2_var,
        constraints['cbf1'].multipliers["roi"],
        constraints['cbf2'].multipliers["roi"],
        clf_epsilon, cbf1_epsilon, cbf2_epsilon,
    ),
    override_symbol_values=set_epsilon_to_zero,
)

In [ ]:
# select solver
solver = sosopt.cvx_opt_solver
# solver = sosopt.mosek_solver

# CVXOPT solver options
#######################

cvxopt.solvers.options['show_progress'] = False
# cvxopt.solvers.options['show_progress'] = True

# cvxopt.solvers.options['maxiters'] = 100

@do()
def solve_problem(step: Step, iter_data: IterationData):
    problem = sosopt.sos_problem(
        lin_cost=step.lin_cost,
        quad_cost=step.quad_cost,
        constraints=constraints.values(),
        solver=solver,
    )

    # overwrite values
    symbol_values = iter_data.symbol_values | step.override_symbol_values

    substitutions = {
        symbol: symbol_values[symbol] for param in step.substitutions for symbol in param.to_symbols()
    }
    
    # filter values that need to be substituted
    problem = problem.eval(substitutions)

    # # print the decision variables for each constraint
    # for primitive in problem.constraint_primitives:
    #     print(f'{primitive.name=}, {primitive.decision_variable_symbols=}')
    
    # solve SOS problem
    sos_result = yield from problem.solve()

    solver_data = sos_result.solver_data
    print(f'{solver_data.status=}, {solver_data.iterations=}, {solver_data.cost=}')
    
    # update iteration data
    n_iter_data = replace(
        iter_data, 
        symbol_values=symbol_values | sos_result.symbol_values, 
        solver_data=solver_data,
    )
    
    epsilon_roi = (clf_epsilon, cbf1_epsilon, cbf2_epsilon, epsilon_roi_V, epsilon_roi_B)
    print(f'epsilon = {tuple(n_iter_data.symbol_values[e.symbol] for e in epsilon_roi)}')

    return statemonad.from_(n_iter_data)

## Run Alternating Algorithm

In [ ]:
for idx in range(1):
    print(f'iteration: {idx}')
        
    if 1e-6 < iter_data.symbol_values[epsilon_roi_B.symbol][0]:
        state, iter_data = solve_problem(step_1_margin, iter_data).apply(state)
        state, iter_data = solve_problem(step_1_roiB, iter_data).apply(state)
    
    if 1e-6 < iter_data.symbol_values[epsilon_roi_V.symbol][0]:
        state, iter_data = solve_problem(step_2_margin, iter_data).apply(state)
        state, iter_data = solve_problem(step_2_roiV, iter_data).apply(state)

        state, iter_data = solve_problem(step_1_margin, iter_data).apply(state)
        state, iter_data = solve_problem(step_1_roiV, iter_data).apply(state)
    else:
        break
    
    if 1e-6 < iter_data.symbol_values[epsilon_roi_B.symbol][0]:
        state, iter_data = solve_problem(step_2_margin, iter_data).apply(state)
        state, iter_data = solve_problem(step_2_roiB, iter_data).apply(state)

In [ ]:
# maximize a surrogate volume of the safe set until the primal cost of the optimization problem decreases by less than 1 percent

# the primal cost of the previous iteration
previous_cost = None

if not (
    1e-6 < iter_data.symbol_values[epsilon_roi_V.symbol][0]
    or 1e-6 < iter_data.symbol_values[epsilon_roi_B.symbol][0]
):
    for idx in range(5):
        print(f'iteration: {idx}')
    
        state, iter_data = solve_problem(step_2_margin, iter_data).apply(state)
        state, iter_data = solve_problem(step_1_vol, iter_data).apply(state)

        if previous_cost is not None:
            if (previous_cost - iter_data.solver_data.cost) < 0.01 * iter_data.solver_data.cost:
                break
        previous_cost = iter_data.solver_data.cost

In [ ]:
def save_symbol_values(arg):
    file_name = '2_symbol_values.p'
    
    with open(file_name, 'wb') as file:   
        pickle.dump(iter_data.symbol_values, file)

# Create a button to ensure that the file is not overritten by accident.
button_download = widgets.Button(description = 'Save symbol values')   
button_download.on_click(save_symbol_values)
display(button_download)

In [ ]:
@do()
def debugging_tools():

    # select the function to be evaluated
    expr = constraints['clf'].multipliers["V"]
    # expr = V

    func = yield polymat.to_array(expr.eval(iter_data.symbol_values), x)
    x = np.array((0.1, 0, 0)).reshape(-1, 1)

    # evaluates the function at x
    print(f'{func(x)=}')

    return statemonad.from_(None)

_ = debugging_tools().apply(state)

## Plot results

This subsection visualizes the results through plots.

In [ ]:
@do()
def plot_result(symbol_values):

    # Helper function to project the 3 dimensional state onto 2 dimensions
    def map_to_xy(x, y):
        return np.array((x, y) + (0,) * (n_states - 2)).reshape(-1, 1)
    
    pyplot.close()
    fig = pyplot.figure(figsize=(8, 8))
    ax = fig.subplots()

    # Create stream plot
    ####################
    
    x_min, x_max, y_min, y_max = -0.8, 0.2, -1.3, 1.3
    args = {'color': 'r', 'linestyle': 'dashed', 'dashes': (5, 5), 'linewidth': 0.8}
    ax.plot(np.array((x_min, x_max)), np.array((y_min, y_min)), **args)
    ax.plot(np.array((x_min, x_max)), np.array((y_max, y_max)), **args)
    ax.plot(np.array((x_min, x_min)), np.array((y_min, y_max)), **args)
    ax.plot(np.array((x_max, x_max)), np.array((y_min, y_max)), **args)


    # # Create stream plot
    # ####################

    f_array = yield from polymat.to_array(f, x)
    G_array = yield from polymat.to_array(G, x)
    p_array = yield from polymat.to_array(p.eval(symbol_values), x)
    s_array = yield from polymat.to_array(s.eval(symbol_values), x)
    
    def get_x_dot(x):
        x = np.array(x).reshape(-1, 1)
        u = p_array(x) / s_array(x)
        xdot = f_array(x) + G_array(x) @ u
        return np.squeeze(xdot)

    ticksX = np.arange(-0.8, 0.2, 0.04)
    ticksY = np.arange(-1.3, 1.34, 0.04)
    n_row, n_col = len(ticksY), len(ticksX)
    X = np.matlib.repmat(ticksX, n_row, 1)
    Y = np.matlib.repmat(ticksY.reshape(-1, 1), 1, n_col)

    stream_U = np.zeros((n_row, n_col))
    stream_V = np.zeros((n_row, n_col))
    def create_stream_data():
        for row, (x_row, y_row) in enumerate(zip(X, Y)):
            for col, (x, y_val) in enumerate(zip(x_row, y_row)):
                u, v, _ = get_x_dot(map_to_xy(x, y_val))
                stream_U[row, col] = u
                stream_V[row, col] = v

    create_stream_data()

    ax.streamplot(X, Y, stream_U, stream_V, density=[0.5, 0.7])

    # Plot Sublevel sets
    ####################
    
    ticks = np.arange(-2.1, 2.1, 0.04)
    X = np.matlib.repmat(ticks, len(ticks), 1)
    Y = X.T

    V_array = yield from polymat.to_array(V.eval(symbol_values), x)
    ZV = np.vectorize(lambda x, y: V_array(map_to_xy(x, y)))(X, Y)
    ax.contour(X, Y, ZV, [0.0], linewidths=2, colors=['#17202A'])

    B1_array = yield from polymat.to_array(B1.eval(symbol_values), x)
    ZB1 = np.vectorize(lambda x, y: B1_array(map_to_xy(x, y)))(X, Y)
    ax.contour(X, Y, ZB1, [0.0], linewidths=0.5, colors=['#A0B1BA'])

    B2_array = yield from polymat.to_array(B2.eval(symbol_values), x)
    ZB2 = np.vectorize(lambda x, y: B2_array(map_to_xy(x, y)))(X, Y)
    ax.contour(X, Y, ZB2, [0.0], linewidths=0.5, colors=['#A0B1BA'])
    
    def select_greater(x, y):   
        v1 = B1_array(map_to_xy(x, y))
        v2 = B2_array(map_to_xy(x, y))

        return v2 if v1 < v2 else v1
        
    Zpick = pick_closest_vec = np.vectorize(select_greater)(X, Y)
    CS = ax.contour(X, Y, Zpick, levels=[0], linewidths=2, colors=['#17202A'])

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)

    ax.set_xlabel(r'${\tilde v}_{dc}$ [p.u.]')
    ax.set_ylabel(r'${\tilde i}_{d}$ [p.u.]')
    
    pyplot.show()

    return statemonad.from_(fig)

state, fig = plot_result(iter_data.symbol_values).apply(state)

In [ ]:
def save_figure(arg):
    fig.savefig('2_stream_plot.pdf', bbox_inches='tight')

# Create a button to ensure that the file is not overritten by accident.
button_download = widgets.Button(description = 'Save figure')   
button_download.on_click(save_figure)
display(button_download)

## Pickle Results

This subsection saves the results in a pickle file, enabling further simulations and analysis.

In [ ]:
def save_arrays(arg):
    @do()
    def gen_arrays(symbol_values):
        symbol_values = iter_data.symbol_values

        s_array = yield from polymat.to_array(s.eval(symbol_values), x)
        
        V_array = yield from polymat.to_array(V.eval(symbol_values), x)
        B1_array = yield from polymat.to_array(B1.eval(symbol_values), x)
        B2_array = yield from polymat.to_array(B2.eval(symbol_values), x)

        dV_array = yield from polymat.to_array(dV.eval(symbol_values), x)
        dB1_array = yield from polymat.to_array(dB1.eval(symbol_values), x)
        dB2_array = yield from polymat.to_array(dB2.eval(symbol_values), x)

        gV_array = yield from polymat.to_array(constraints['clf'].multipliers['V'].eval(symbol_values), x)
        gB1_array = yield from polymat.to_array(constraints['cbf1'].multipliers['B'].eval(symbol_values), x)
        gB2_array = yield from polymat.to_array(constraints['cbf2'].multipliers['B'].eval(symbol_values), x)

        arrays = {
            's': s_array,
            'V': V_array,
            'B1': B1_array,
            'B2': B2_array,
            'dV': dV_array,
            'dB1': dB1_array,
            'dB2': dB2_array,
            'gV': gV_array,
            'gB1': gB1_array,
            'gB2': gB2_array,
        }

        return statemonad.from_(arrays)

    _, arrays = gen_arrays(iter_data.symbol_values).apply(state)

    file_name = '2_arrays.p'

    with open(file_name, 'wb') as file:   
        pickle.dump(arrays, file)

# Create a button to ensure that the file is not overritten by accident.
button_download = widgets.Button(description = 'Save arrays')   
button_download.on_click(save_arrays)
display(button_download)